In [ ]:
import cv2
import numpy as np

# load image
img = cv2.imread("building.png")
# add blur because of pixel artefacts 
img = cv2.GaussianBlur(img, (5, 5), 5)
# convert to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 

# set lower and upper color limits
# lower_val = (90, 50, 20)
# upper_val = (110, 100, 255) # building glass
lower_val = (0, 0, 20)
upper_val = (180,15,255) # grey
# lower_val = (95, 0, 0)
# upper_val = (100,25,255)

# Threshold the HSV image to get only green colors
mask = cv2.inRange(hsv, lower_val, upper_val)
# apply mask to original image
res = cv2.bitwise_and(img, img, mask=mask)
cv2.imshow('Result', res)
# print(res)
print(len(res[np.where(res != 0)]))

# detect contours in image
contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# draw filled contour on result
for cnt in contours:
    cv2.drawContours(res, [cnt], 0, (0,0,255), 2)
print(len(res[np.where(res != 0)]))
cv2.imshow("resu", res)
    
# detect edges in mask
edges = cv2.Canny(mask,100,100)
# print(edges)

# # to save an image use cv2.imwrite('filename.png',img)
# #show images
# cv2.imwrite("../goal_images/Result_with_contours.png", res)
# cv2.imshow("../goal_images/Mask.png", mask)
# cv2.imshow("Edges.png", res)
cv2.waitKey(0)
cv2.destroyAllWindows()

1167360
1058249


In [ ]:
import cv2
import numpy as np

img = cv2.imread('building.png')
# img = cv2.imread('../goal_images/build2.png')
# img = cv2.imread('../goal_images/street3.png')

brightness = 0
contrast = 10
brightness += int(round(255*(1-contrast)/2))
img = cv2.addWeighted(img, contrast, img, 0, brightness)
cv2.imshow("contract", img)

img = cv2.GaussianBlur(img, (5, 5), 5)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 

lower_val = (0, 0, 20)
upper_val = (180,15,255) # grey
# lower_val = (90, 50, 20)
# upper_val = (110, 100, 255) # building glass

mask = cv2.inRange(hsv, lower_val, upper_val)
img = cv2.bitwise_and(img, img, mask=mask)
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# ret,thresh = cv2.threshold(img,50,255,0)
# contours,hierarchy = cv2.findContours(img, 1, 2)
contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print("Number of contours detected:", len(contours))
count = 0
areas = []
for cnt in contours:
    x1,y1 = cnt[0][0]
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
#     print(len(approx))
   
    if len(approx) > 4:
        count+=1
        ar = cv2.contourArea(cnt)
        areas.append(ar)
        x, y, w, h = cv2.boundingRect(cnt)
        ratio = float(w)/h
        if ratio >= 0.9 and ratio <= 1.1:
            img = cv2.drawContours(img, [cnt], -1, (0,255,255), 3)
        else:
            img = cv2.drawContours(img, [cnt], -1, (0,255,0), 3)
print(count)
print(sorted(areas, reverse=True))
cv2.imshow("Shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of contours detected: 69
25
[31568.5, 326.5, 79.0, 53.5, 29.0, 24.5, 17.0, 17.0, 15.5, 13.5, 13.5, 11.5, 9.5, 7.5, 7.0, 6.5, 6.0, 5.5, 4.5, 3.5, 3.0, 2.5, 2.5, 1.5, 1.5]


In [ ]:
20,853 - street
3,36,969 - same lane
31,568.5 - diff lane

19,779 - street
2,70,722.5 - same lane
31242 - diff_lane


In [ ]:
import cv2
import numpy as np


# img = cv2.imread('building.png')
img = cv2.imread('../goal_images/build1.png')
# img = cv2.imread('../goal_images/street3.png')

image_copy = img.copy() 
height = img.shape[0]
width = img.shape[1]

def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    # Retrieve the number of color channels of the image.
    channel_count = img.shape[2]
    # Create a match color with the same color channel counts.
    match_mask_color = (255,) * channel_count
      
    # Fill inside the polygon
    cv2.fillPoly(mask, vertices, match_mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

region_of_interest_vertices = [
    (0, 0),
    (width, 0),
    (width, 2*height/3),
    (width/2, height),
    (0, 2*height/3),
]

roi = region_of_interest(
    image_copy,
    np.array([region_of_interest_vertices], np.int32),
)

cv2.imshow("cropped", roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

first = cv2.imread('goal_tile_images/duck/duck_map1_0_72.png')
second = cv2.imread('goal_tile_images/noDuck/noDuck_map1_0_2.png')

# Convert images to grayscale
first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
second_gray = cv2.cvtColor(second, cv2.COLOR_BGR2GRAY)

# Compute SSIM between two images
score, diff = structural_similarity(first_gray, second_gray, full=True)
print("Similarity Score: {:.3f}%".format(score * 100))

Similarity Score: 65.660%


In [11]:
from os import listdir
from os.path import isfile, join

mypath = 'goal_tile_images/duck'
map_name = 'map1_1'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and map_name in f]
onlyfiles

['duck_map1_1_68.png',
 'duck_map1_1_69.png',
 'duck_map1_1_62.png',
 'duck_map1_1_63.png',
 'duck_map1_1_67.png',
 'duck_map1_1_66.png',
 'duck_map1_1_64.png',
 'duck_map1_1_65.png']